<a href="https://colab.research.google.com/github/SinghPrach/30-days-with-python/blob/main/GoogleColab_ATS_ResumeScanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://oindrilasen.com/2021/05/build-resume-scanner-using-python-nlp/

In [7]:
!pip install pdfminer

In [8]:
!pip install docx2txt

In [9]:
import io
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage
import docx2txt
import re
import operator
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
set(stopwords.words('english'))
from wordcloud import WordCloud
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Read PDF Resume

In [10]:
def read_pdf_resume(pdf_doc):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    with open(pdf_doc, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
            text = fake_file_handle.getvalue()
    converter.close()
    fake_file_handle.close()
    if text:
        return text

Read Word Resume

In [11]:
def read_word_resume(word_doc):
     resume = docx2txt.process(word_doc)
     resume = str(resume)
     #print(resume)
     text =  ''.join(resume)
     text = text.replace("\n", "")
     if text:
         return text

In [12]:
resume_w = read_word_resume('/content/Resume/PrachiSingh.docx')

In [13]:
print(resume_w)

PRACHI SINGH Bangalore, Karnataka-560038, India | 6204607306 | singh.indiap@gmail.com | Indiahttps://github.com/SinghPrach/SinghPrach/blob/main/README.mdPROFESSIONAL SUMMARY Detail-oriented Big Data Developer with 2 plus years of experience and a strong background in Data Engineering, Data Manipulation, Data Analysis and data driven strategies by presenting actionable data. Skilled in using Python and SQL and experience in developing ETL pipelines and demonstrated ability in participating in discussions with the business stakeholders. WORK EXPERIENCE Data Engineer, PwC India 				                                                                      July 2022 - PresentDesigned and implemented end-to-end data pipelines for processing large datasets of different file formats (csv, parquet,xslx,xlsb,sql tables), resulting in a 75% improvement in data processing efficiency.Developed a planning tool (optimiser) in Pyspark and python for an FMCG, that resulted in an increase in Gross Profit of

In [14]:
def clean_job_decsription(jd):
     ''' a function to create a word cloud based on the input text parameter'''
     ## Clean the Text
     # Lower
     clean_jd = jd.lower()
     # remove punctuation
     clean_jd = re.sub(r'[^\w\s]', '', clean_jd)
     # remove trailing spaces
     clean_jd = clean_jd.strip()
     # remove numbers
     clean_jd = re.sub('[0-9]+', '', clean_jd)
     # tokenize
     clean_jd = word_tokenize(clean_jd)
     # remove stop words
     stop = stopwords.words('english')
     clean_jd = [w for w in clean_jd if not w in stop]
     return(clean_jd)

In [15]:
job_description_text = """Bachelor's degree, OR 3+ years of relevant work experience
Preferred Qualifications
2 yrs. work experience with a bachelor’s degree or master’s or advanced degree in an analytical field such as computer science, statistics, finance, economics or relevant area.
Working knowledge of Hadoop ecosystem and associated technologies - Hadoop, Hive, Spark
Experience in writing Spark and Hive code to process large data sets in Hadoop environments
Experience with SQL for extracting, aggregating, and processing big data using Hadoop
Development experience in one or more of the following: Scala, Python or Java.
Basic understanding of RDBMs viz, MS SQL, DB2, Oracle, etc for data retrieval
Experience using VCS like Git
Understanding of Kafka, Apache Hudi will be a plus."""

In [16]:
clean_jd = clean_job_decsription(job_description_text)
print(clean_jd)

['bachelors', 'degree', 'years', 'relevant', 'work', 'experience', 'preferred', 'qualifications', 'yrs', 'work', 'experience', 'bachelors', 'degree', 'masters', 'advanced', 'degree', 'analytical', 'field', 'computer', 'science', 'statistics', 'finance', 'economics', 'relevant', 'area', 'working', 'knowledge', 'hadoop', 'ecosystem', 'associated', 'technologies', 'hadoop', 'hive', 'spark', 'experience', 'writing', 'spark', 'hive', 'code', 'process', 'large', 'data', 'sets', 'hadoop', 'environments', 'experience', 'sql', 'extracting', 'aggregating', 'processing', 'big', 'data', 'using', 'hadoop', 'development', 'experience', 'one', 'following', 'scala', 'python', 'java', 'basic', 'understanding', 'rdbms', 'viz', 'ms', 'sql', 'db', 'oracle', 'etc', 'data', 'retrieval', 'experience', 'using', 'vcs', 'like', 'git', 'understanding', 'kafka', 'apache', 'hudi', 'plus']


In [17]:
def create_word_cloud(jd):
     corpus = jd
     fdist = FreqDist(corpus)
     #print(fdist.most_common(100))
     words = ' '.join(corpus)
     words = words.split()
     data = dict()
     for word in (words):
      word = word.lower()
      data[word] = data.get(word, 0) + 1
      dict(sorted(data.items(), key=operator.itemgetter(1),reverse=True))
      word_cloud = WordCloud(width = 800, height = 800,background_color ='white',max_words = 500)
      word_cloud.generate_from_frequencies(data)
      plt.figure(figsize = (10, 8), edgecolor = 'k')
      plt.imshow(word_cloud,interpolation = 'bilinear')
      plt.axis("off")
      plt.tight_layout(pad = 0)
      plt.show()

In [18]:
def get_resume_score(text):
    cv = CountVectorizer(stop_words='english')
    count_matrix = cv.fit_transform(text)
    #Print the similarity scores
    print("\nSimilarity Scores:")

    #get the match percentage
    matchPercentage = cosine_similarity(count_matrix)[0][1] * 100
    matchPercentage = round(matchPercentage, 2) # round to two decimal

    print("Your resume matches about "+ str(matchPercentage)+ "% of the job description.")

In [20]:
# resume = read_pdf_resume('/content/Resume/PrachiSingh.pdf')

job_description = input("\nEnter the Job Description: ")
    ## Get a Keywords Cloud
clean_jd = clean_job_decsription(job_description)
# create_word_cloud(clean_jd)


Enter the Job Description: 


In [22]:
text = [resume_w, job_description_text]
    ## Get a Match score
get_resume_score(text)


Similarity Scores:
Your resume matches about 35.21% of the job description.


In [ ]:
# For the same job description, same resume got two different scores, when entered in 2 different formats.
# 1. .pdf and 2. .docx
# In pdf format, the score was 6.82%
# In docx format, the score was 35.21%
#### Conclusion: While applying, always apply in the .docx format